In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import random

In [2]:
tags_animes = pd.read_csv(os.path.join("data", "tags_animes.csv"))

In [3]:
detalles_animes = pd.read_csv(os.path.join("data", "MAL_dataframe.csv"))

In [4]:
# Convertir los posibles valores NaN de Tags a "" (caso hipótetico)

tags_animes["Tags"] = tags_animes["Tags"].fillna("")

In [5]:
# Será conveniente renombrar MAL_ID a id para poder hacer un merge con el dataframe de
# detalles_animes

tags_animes = tags_animes.rename(columns={"MAL_ID": "id"})

In [6]:
# Obtener todos los tags únicos en el DataFrame

all_tags = set(
    tag.strip() if isinstance(tags_str, str) else tag
    for tags_str in tags_animes["Tags"]
    for tag in (
        tags_str.strip("[]").split(",") if isinstance(tags_str, str) else tags_str
    )
)


# Además, quitar las ' de los tags, que no sirven de nada, y están al comienzo y al final de cada
# tag

all_tags = set(tag.strip("'") for tag in all_tags)

# Eliminar el tag fantasma de all_tags, cuyo valor es ""; no sirve de nada la verdad.
try:
    all_tags.remove("")
except KeyError:
    pass
try:
    all_tags.remove(" ")
except KeyError:
    pass
try:
    all_tags.remove("")
except KeyError:
    pass
try:
    all_tags.remove(" ")
except KeyError:
    pass
try:
    all_tags.remove("  ")
except KeyError:
    pass

# Crear columnas para cada tag en el DataFrame
for tag in all_tags:
    tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)

C:\Users\mativ\AppData\Local\Temp\ipykernel_50088\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)
C:\Users\mativ\AppData\Local\Temp\ipykernel_50088\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)
C:\Users\mativ\AppData\Local\Temp\ipykernel_50088\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [7]:
# Por conveniencia, lo más útil es eliminar la columna "URL", junto a "Tags" y "Alts", que ya no
# sirven de nada. Además se establece la columna "MAL_ID" como índice.

tags_animes = tags_animes.drop(columns=["URL", "Tags", "Alts", "Anime"]).set_index("id")

In [8]:
# El MAL_ID de índice puede conectarse con los datos de detalles_animes, que también tienen MAL_ID,
# y así se puede obtener que el índice sea el title de cada anime, que es más útil y fácil de
# visualizar.

tags_animes = tags_animes.join(detalles_animes.set_index("id")["title"])

In [9]:
# Cambio de índice a title, eliminación del índice actual igualmente.

tags_animes = tags_animes.set_index("title")

In [10]:
# Por temas de memoria principalmente, es conveniente hacer el dataframe sparse.

tags_animes = tags_animes.astype(pd.SparseDtype(np.int8, fill_value=0))

In [11]:
tags_animes 

,Romance,Borderline H,Pandemic,Mountain Climbing,Afterlife,Framed for a Crime,Age Gap,Transported Into a Novel,Aquarium,Earthquake,...,Tournaments,Germany,Submarine,Obsessive Love,Feudal Japan,World War 2,Flash Animation,Samurai,Mental Illness,Anime no Chikara
title,,,,,,,,,,,,,,,,,,,,,
"""0""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Memoire",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Movie",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Eiji""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuori Qing Kong,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zurui Maboroshi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zutto Mae kara Suki deshita. Kokuhaku Jikkou Iinkai,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
with open(os.path.join("data", "users_df.pickle"), "rb") as archivo:
    users_df = pickle.load(archivo)

In [13]:
users_df

,39110:xxbladexx20,216949:haijuta,246587:d3athzero,107198:JayJay1401,45036:Seleare,4328:Joseph_,110895:Rudenick,93256:Moltke,160129:dander,256598:melandrea,...,16805797:iambabymango,16809902:Flopoflop,16791488:RhaenyraFR,16838607:SasOnator,17063891:Dezwhite05,17197026:dumb_zoro_,16617450:kotyboh,16459755:Tiberjuice,16965619:Watcher794_2,16656371:Aoto_uchiha10
24,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,8.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8978 entries, 24 to 33183
Columns: 7793 entries, 39110:xxbladexx20 to 16656371:Aoto_uchiha10
dtypes: Sparse[float64, nan](7793)
memory usage: 6.5 MB


In [15]:
# Convertimos asimismo el tipo de dato hacia float64, dado es lo necesario para realizar gráficos
# correctamente (testeado)

users_df = users_df.astype(pd.SparseDtype(np.float32, fill_value=np.nan))

Dado que tenemos demasiadas columnas de información, lo que haremos sea lo siguiente para unificar
todo para el clustering. 

En primer lugar, de acuerdo a cada tag, veremos cuánta puntación sumada le da cualquier usuario a
ese tag, considerando todos los animes que ha visto. Considerar que el nombre del anime se
encuentra de forma igualmente en detalles_anime